In [1]:
# py파일 또는 ipynb 파일 새로 생성하여 진행
import json

with open('bank4.json','r') as f:
    data = json.load(f)

In [2]:
from IPython.display import display, JSON
display(data)

{'1c': {'c_id': '1c',
  'name': '최원칠',
  'account_num': 2,
  'total_amount': 500,
  'rat': 'silver',
  'account': {'11a': {'a_id': '11a',
    'password': '11p',
    'c_id': '1c',
    'amount': 100},
   '12a': {'a_id': '12a', 'password': '12p', 'c_id': '1c', 'amount': 400}}},
 '2c': {'c_id': '2c',
  'name': '이서혁',
  'account_num': 1,
  'total_amount': 2000,
  'rat': 'gold',
  'account': {'21a': {'a_id': '21a',
    'password': '21p',
    'c_id': '2c',
    'amount': 2000}}},
 '3c': {'c_id': '3c',
  'name': 'aiot',
  'account_num': 3,
  'total_amount': 50000,
  'rat': 'vip',
  'account': {'31a': {'a_id': '31a',
    'password': '31p',
    'c_id': '3c',
    'amount': 10000},
   '32a': {'a_id': '32a', 'password': '32p', 'c_id': '3c', 'amount': 20000},
   '33a': {'a_id': '33a', 'password': '33p', 'c_id': '3c', 'amount': 20000}}},
 '4c': {'c_id': '4c',
  'name': '새싹',
  'account_num': 4,
  'total_amount': 200000,
  'rat': 'vvip',
  'account': {'41a': {'a_id': '41a',
    'password': '41p',
    '

In [3]:
# 파이어스토어 db 인스턴스 생성
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

# Use a service account.
cred = credentials.Certificate('./nugunaaiot-maeng-1004a11a5af7.json')

app = firebase_admin.initialize_app(cred)

db = firestore.client()

In [5]:
doc_ref = db.collection('customer').document('c1')

doc = doc_ref.get()

if doc.exists:
    print(f'Document data: {display(doc.to_dict())}')
else:
    print(u'No such document!')

{'name': '최원칠',
 'account_num': 2,
 'c_id': 'c1',
 'edit_time': DatetimeWithNanoseconds(2022, 12, 10, 7, 47, 47, 272000, tzinfo=datetime.timezone.utc),
 'rat': 'silver',
 'total_amount': 900,
 'accounts': {'11a': {'c_id': 'c1', 'amount': 100, 'a_id': '11a'},
  '12a': {'c_id': 'c1', 'amount': 400, 'a_id': '12a'}}}

Document data: None


In [6]:
# bank4.json 파일내의 데이터를 파이어스토어 customer 컬렉션에 고객 아이디를 아이디로 가지는 문서들로 추가
users_ref = db.collection('customer')
docs = users_ref.stream()

print(docs)

<generator object Query.stream at 0x000002337DE95F50>


In [15]:
for key in data.keys():
    doc_ref = db.collection('customer').document(key)
    doc_ref.set(data[key])

In [16]:
# 데이터 잘 들어갔는지 체크 전체 읽기
users_ref = db.collection('customer')
docs = users_ref.stream()

for doc in docs:
    print(f'{doc.id} => {doc.to_dict()}')

1c => {'account': {'11a': {'password': '11p', 'c_id': '1c', 'amount': 100, 'a_id': '11a'}, '12a': {'password': '12p', 'c_id': '1c', 'amount': 400, 'a_id': '12a'}}, 'c_id': '1c', 'rat': 'silver', 'total_amount': 500, 'account_num': 2, 'name': '최원칠'}
2c => {'account': {'21a': {'password': '21p', 'c_id': '2c', 'amount': 2000, 'a_id': '21a'}}, 'account_num': 1, 'rat': 'gold', 'total_amount': 2000, 'c_id': '2c', 'name': '이서혁'}
3c => {'account': {'31a': {'password': '31p', 'c_id': '3c', 'amount': 10000, 'a_id': '31a'}, '32a': {'password': '32p', 'c_id': '3c', 'amount': 20000, 'a_id': '32a'}, '33a': {'password': '33p', 'c_id': '3c', 'amount': 20000, 'a_id': '33a'}}, 'account_num': 3, 'rat': 'vip', 'total_amount': 50000, 'c_id': '3c', 'name': 'aiot'}
4c => {'account': {'44a': {'password': '44p', 'c_id': '4c', 'amount': 30000, 'a_id': '44a'}, '41a': {'password': '41p', 'c_id': '4c', 'amount': 100000, 'a_id': '41a'}, '42a': {'password': '42p', 'c_id': '4c', 'amount': 50000, 'a_id': '42a'}, '43a'

In [17]:
# {아이디: 5c, 이름: 본인이름, 계좌개수:0, 총금액:0, 등급:normal, 계좌:비어있는 딕셔너리} 고객 customer 컬렉션에 추가
doc_ref = db.collection('customer').document('5c')
doc_ref.set({
    'c_id': '5c',
    'name': '맹광국',
    'account_num': 0,
    'total_amount': 0,
    'rat': 'silver',
    'account': {}
})

update_time {
  seconds: 1670659955
  nanos: 629041000
}

In [22]:
# 5c 고객에 임의의 계좌 추가 (고객의 계좌, 총금액, 계좌개수, 등급, 수정시간 업데이트되어야함)
# 계좌개수, 총금액은 firestore.Increment() 사용
doc_ref = db.collection('customer').document('5c')
doc_ref.update({'account_num': firestore.Increment(1),# +=
            'total_amount' : firestore.Increment(500),
            'edit_time': firestore.SERVER_TIMESTAMP, # 숨겨진 칼럼으로 fb기능을 사용가능
            'accounts.1a':{'a_id':'1a', 'c_id':'c1', 'amount':500} # 콜렉션.필드id 를 키값으로 사용가능
               })

doc_ref = db.collection('customer').document('5c')

doc = doc_ref.get()
if doc.exists:
    print(f'Document data: {display(doc.to_dict())}')
else:
    print(u'No such document!')

{'edit_time': DatetimeWithNanoseconds(2022, 12, 10, 8, 20, 8, 233000, tzinfo=datetime.timezone.utc),
 'account_num': 2,
 'total_amount': 1000,
 'accounts.1a': {'c_id': 'c1', 'amount': 500, 'a_id': '1a'},
 'accounts': {'1a': {'c_id': 'c1', 'amount': 500, 'a_id': '1a'}}}

Document data: None
